# Part 1

In [1]:
import random
import numpy
import math
import pandas as pd 

## Generate Distribution Functions

In [2]:

def sorted_indexes(payoffs):
    vals_indexes=[]
    ind_by_val = []

    for i in range(len(payoffs)):
        vals_indexes.append([payoffs[i],i])
    
    vals_indexes.sort(reverse=True)
    for x in vals_indexes:
        ind_by_val.append(x[1])
    return ind_by_val

def find_min_index(payoffs):
    min_value = min(payoffs)
    min_index = payoffs.index(min_value)
    return min_index


def generate_adversarial_payoffs(num_actions, num_rounds):
    rounds_list = []
    totals_by_round = []
    initial_payoff = round(random.random(), 2)
    first_payoffs = [0 for i in range(num_actions)]
    first_payoffs[random.randrange(num_actions)] = initial_payoff
    total_payoffs = [first_payoffs[i] for i in range(num_actions)]
    min_index = find_min_index(total_payoffs)
    rounds_list.append(first_payoffs)
    totals_by_round.append([total_payoffs[i] for i in range(num_actions)])
    
    for i in range(num_rounds - 1):
        new_payoff = round(random.random(), 2)
        adversarial_payoffs = [0 for i in range(num_actions)]
        adversarial_payoffs[min_index] = new_payoff
        for i in range(num_actions):
            total_payoffs[i] += adversarial_payoffs[i]
            total_payoffs[i] = round(total_payoffs[i], 2)
        
        min_index = find_min_index(total_payoffs)
        new_totals = [total_payoffs[i] for i in range(num_actions)]
        totals_by_round.append(new_totals)
        rounds_list.append(adversarial_payoffs)

    #print("utility at each round: \n", rounds_list)
    #print("totals by round: \n", totals_by_round)
    #print("final payoffs: \n", total_payoffs)
    return rounds_list, totals_by_round


#generate_adversarial_payoffs(10, 10)

In [3]:
#when generating the bernoulli payoffs, generate the payoffs of each action at each round and the 
#total payoffs up to that point for each action. i.e. list of lists of payoffs/round & list of lists of aggregated payoffs.
#uncomment the last line of the generate_adversarial_payoffs section for an example
def find_payoff(success_chance):
    comparison_val = random.random()
    return int(success_chance > comparison_val)

def generate_bernoulli_payoffs(num_actions, num_rounds):
    rounds_list = []
    totals_by_round = []
    total_payoffs = [0 for i in range(num_actions)]
    totals_by_round = []
    action_success_chances = [round(random.random() / 2,2) for i in range(num_actions)]
    
    for i in range(num_rounds):
        new_payoffs = [find_payoff(action_success_chances[j]) for j in range(num_actions)]
        
        for i in range(num_actions):
            total_payoffs[i] += new_payoffs[i]
            total_payoffs[i] = round(total_payoffs[i], 2)
        
        new_totals = [total_payoffs[i] for i in range(num_actions)]
        totals_by_round.append(new_totals)
        rounds_list.append(new_payoffs)

    #print("utility at each round: \n", rounds_list)
    #print("totals by round: \n", totals_by_round)
    #print("final payoffs: \n", total_payoffs)
    return rounds_list, totals_by_round

#generate_bernoulli_payoffs(3, 3)



In [4]:
def rotate_action_payoffs(action_payoffs):
    copy_payoffs = [action_payoffs[i] for i in range(len(action_payoffs))]
    for i in range(0, len(action_payoffs)):
        action_payoffs[i] = copy_payoffs[i-1]
    return action_payoffs

def generate_rotational_random_payoffs(num_actions, num_rounds):
    rounds_list = []
    totals_by_round = []
    action_payoffs = [round(random.random(), 2) for i in range(num_actions)]
    total_payoffs = [0 for i in range(num_actions)]
    
    for i in range(num_rounds):
        if random.random() > 0.9:
            action_payoffs = rotate_action_payoffs(action_payoffs)
            
        for i in range(num_actions):
            total_payoffs[i] += action_payoffs[i]
            total_payoffs[i] = round(total_payoffs[i], 2)
        new_totals = [total_payoffs[i] for i in range(num_actions)]    
        rounds_list.append([action_payoffs[i] for i in range(num_actions)])
        totals_by_round.append(new_totals)
        
    return rounds_list, totals_by_round


## Simulate Algorithm Behavior Functions

In [5]:
def simulate_exponential_weights(rounds_list, totals_by_round, epsilon, max_payoff):
    num_rounds = len(rounds_list)
    num_actions = len(rounds_list[0])
    choices_made = []
    action_weights = []
    action_probabilities = [(1/num_actions) for i in range(num_actions)]
    current_weights = [1 for i in range(num_actions)]
    action_weights.append(current_weights)
    alg_payoffs = []
    opt_payoffs = []
    
    for round in range(1, num_rounds):
        last_round = round - 1
        current_weights = [None for i in range(num_actions)]
        for action in range(num_actions):
            V_last = totals_by_round[last_round][action]
            exp = V_last / max_payoff
            current_weights[action] = pow(1 + epsilon, exp)
        #randomly select from actions using weights as probabilities
        selected_payoff = random.choices(rounds_list[round], weights=current_weights, k=1)[0]
        alg_payoffs.append(selected_payoff)  
        opt_payoffs.append(max(rounds_list[round]))
        action_weights.append(current_weights)
        
    return alg_payoffs, totals_by_round

## Monte Carlo Trials

- Declare size of inputs
- Generate payoffs
- For each learning rate $\{\epsilon_1, . . ., \epsilon_n\}$
    - For each input
        - Simulate the algorithm
        - calculate OPT (best in hindsight payoff)
        - calculate the algorithm's regret
    - Calculate the average regret for this learning rate $\epsilon$

In [6]:

def sum_to_round_i(alg_payoffs, current_round):
    total = 0
    for i in range(current_round):
        total += alg_payoffs[i]
    return total

def individual_regrets(alg_payoffs, round_totals):
    final_payoffs = round_totals[-1]
    opt_action = final_payoffs.index(max(final_payoffs))
    #print(opt_action)
    individual_regrets = [0 for i in range(len(alg_payoffs))]
    for round in range((len(alg_payoffs))):
        individual_regrets[round] = (round_totals[round][opt_action] - sum_to_round_i(alg_payoffs, round)) / (round + 1)
    return individual_regrets
    

rounds = 100
actions = 5
N = 1000
# ADD OPTIMAL LEARNING RATE EPSILON
opt_lr_eps = math.sqrt(numpy.log(actions)/rounds)
learning_rates = [0, 0.25, 0.5, opt_lr_eps, 0.75, 1, 100]

#adversarial monte carlo trial
max_payoff = 1
avg_lr_payoffs = dict()
all_opt_payoffs = []
avg_regret_per_round = dict()
for n in range(N):
    adversarial_payoffs, adversarial_totals = generate_adversarial_payoffs(actions, rounds)
    for epsilon in learning_rates:
        adv_payoffs, adv_round_totals = simulate_exponential_weights(adversarial_payoffs, adversarial_totals, epsilon, max_payoff)
        adv_regrets = individual_regrets(adv_payoffs, adv_round_totals)
        adv_avg_regrets = sum(adv_regrets) / len(adv_regrets)
        adv_final_regret = adv_regrets[-1]
        if epsilon not in avg_regret_per_round:
            avg_regret_per_round[epsilon] = adv_regrets
        else:
            for i in range(len(avg_regret_per_round[epsilon])):
                avg_regret_per_round[epsilon][i] = ((n * avg_regret_per_round[epsilon][i]) + adv_regrets[i]) / (n + 1)            
        if epsilon not in avg_lr_payoffs:
            avg_lr_payoffs[epsilon] = [sum(adv_payoffs)]
        else:
            avg_lr_payoffs[epsilon].append(sum(adv_payoffs))
    all_opt_payoffs.append(max(adv_round_totals[-1]))
for key, val in avg_regret_per_round.items():
    print("Average ALG regret for epsilon =", key, "on adversarial distribution =", val[-1])

for key, val in avg_lr_payoffs.items():
    print("Average ALG payoff for epsilon =", key, "on adversarial distribution =", sum(val) / len(val))
print("Average OPT payoff for adversarial distribution =", sum(all_opt_payoffs) / len(all_opt_payoffs))  

#bernoulli monte carlo trial
max_payoff = 1
avg_lr_payoffs = dict()
all_opt_payoffs = []
avg_regret_per_round = dict()
for n in range(N):
    bernoulli_payoffs, bernoulli_totals = generate_bernoulli_payoffs(actions, rounds)
    for epsilon in learning_rates:
        bern_payoffs, bern_round_totals = simulate_exponential_weights(bernoulli_payoffs, bernoulli_totals, epsilon, max_payoff)
        bern_regrets = individual_regrets(bern_payoffs, bern_round_totals)
        bern_avg_regrets = sum(bern_regrets) / len(bern_regrets)
        bern_final_regret = bern_regrets[-1]
        if epsilon not in avg_regret_per_round:
            avg_regret_per_round[epsilon] = bern_regrets
        else:
            for i in range(len(avg_regret_per_round[epsilon])):
                avg_regret_per_round[epsilon][i] = ((n * avg_regret_per_round[epsilon][i]) + bern_regrets[i]) / (n + 1)
        
        if epsilon not in avg_lr_payoffs:
            avg_lr_payoffs[epsilon] = [sum(bern_payoffs)]
        else:
            avg_lr_payoffs[epsilon].append(sum(bern_payoffs))
    
    all_opt_payoffs.append(max(bern_round_totals[-1]))
for key, val in avg_regret_per_round.items():
    print("Average ALG regret for epsilon =", key, "on bernoulli distribution =", val[-1])
for key, val in avg_lr_payoffs.items():
    print("Average ALG payoff for epsilon =", key, "on bernoulli distribution =", sum(val) / len(val))
print("Average OPT payoff for bernoulli distribution =", sum(all_opt_payoffs) / len(all_opt_payoffs) )

# rotational generation monte carlo trial
generate_rotational_random_payoffs
max_payoff = 1
avg_lr_payoffs = dict()
all_opt_payoffs = []
avg_regret_per_round = dict()
for n in range(N):
    rotational_payoffs, rotational_totals = generate_rotational_random_payoffs(actions, rounds)
    for epsilon in learning_rates:
        rot_payoffs, rot_round_totals = simulate_exponential_weights(rotational_payoffs, rotational_totals, epsilon, max_payoff)
        rot_regrets = individual_regrets(rot_payoffs, rot_round_totals)
        rot_avg_regrets = sum(rot_regrets) / len(rot_regrets)
        rot_final_regret = rot_regrets[-1]
        if epsilon not in avg_regret_per_round:
            avg_regret_per_round[epsilon] = rot_regrets
        else:
            for i in range(len(avg_regret_per_round[epsilon])):
                avg_regret_per_round[epsilon][i] = ((n * avg_regret_per_round[epsilon][i]) + rot_regrets[i]) / (n + 1)
        
        if epsilon not in avg_lr_payoffs:
            avg_lr_payoffs[epsilon] = [sum(rot_payoffs)]
        else:
            avg_lr_payoffs[epsilon].append(sum(rot_payoffs))
    
    all_opt_payoffs.append(max(rot_round_totals[-1]))
for key, val in avg_regret_per_round.items():
    print("Average ALG regret for epsilon =", key, "on rotational random distribution =", val[-1])
for key, val in avg_lr_payoffs.items():
    print("Average ALG payoff for epsilon =", key, "on rotational random distribution =", sum(val) / len(val))
print("Average OPT payoff for rotational random distribution =", sum(all_opt_payoffs) / len(all_opt_payoffs) )


Average ALG regret for epsilon = 0 on adversarial distribution = 0.0017908080808080793
Average ALG regret for epsilon = 0.25 on adversarial distribution = 0.00809060606060606
Average ALG regret for epsilon = 0.5 on adversarial distribution = 0.012438686868686856
Average ALG regret for epsilon = 0.12686362411795196 on adversarial distribution = 0.005380707070707069
Average ALG regret for epsilon = 0.75 on adversarial distribution = 0.016377373737373722
Average ALG regret for epsilon = 1 on adversarial distribution = 0.0211142424242424
Average ALG regret for epsilon = 100 on adversarial distribution = 0.0801304040404042
Average ALG payoff for epsilon = 0 on adversarial distribution = 9.983989999999993
Average ALG payoff for epsilon = 0.25 on adversarial distribution = 9.340209999999999
Average ALG payoff for epsilon = 0.5 on adversarial distribution = 8.90454
Average ALG payoff for epsilon = 0.12686362411795196 on adversarial distribution = 9.623960000000002
Average ALG payoff for epsilo

# Part 2C. Data in the wild

## Data Cleaning 

In [40]:
#Stock dataset taken from https://www.kaggle.com/datasets/camnugent/sandp500?select=all_stocks_5yr.csv

df = pd.read_csv('all_stocks_5yr.csv') 

df = df.query('date.str.startswith("2017")', 
engine="python")

df['Day']=df.groupby(['Name']).cumcount()+1

#Add payoffs of each stock each day as column 

df['payoff'] = df['close'] / df['open']

#get the number of total stocks
df.drop_duplicates(subset = ["Name"]).shape[0]

#get the number of total stocks
df.drop_duplicates(subset = ["Name"]).shape[0]

#Max payoff value

df['payoff'].max()

#query stocks by date or day
df[df['Day'] == 1]

df

#identify stocks that do not have full year stock prices data

temp = df[df['Day'] == 1]
temp = temp['Name'].tolist()

temp_2 = df[df['Day'] == 251]
temp_2 = temp_2['Name'].tolist()

print('Stocks that do not have data for all dates throughout the year: ', ','.join(set(temp).difference(temp_2)))

#drop outlier stocks with not enough data

df = df.drop(df[df.Name == 'BHF'].index)
df = df.drop(df[df.Name == 'DXC'].index) 
df = df.drop(df[df.Name == 'HLT'].index) 
df = df.drop(df[df.Name == 'APTV'].index) 
df = df.drop(df[df.Name == 'DWDP'].index) 
df = df.drop(df[df.Name == 'BHGE'].index) 

df 

#day = 1
#while day <= 251:
#    temp_ds = df[df['Day'] == day]
#    action_payoffs = temp_ds['payoff'].tolist()
#    day += 1
#    print(len(action_payoffs),'num of stocks')
#        #print(len(total_payoffs),'total_payoffs')
#    print(day)



Stocks that do not have data for all dates throughout the year:  BHF,DXC,HLT,APTV,DWDP,BHGE


,date,open,high,low,close,volume,Name,Day,payoff
982,2017-01-03,47.28,47.340,46.135,46.30,6737752,AAL,1,0.979272
983,2017-01-04,46.63,47.435,46.350,46.70,5859604,AAL,2,1.001501
984,2017-01-05,46.52,46.930,45.610,45.89,6825316,AAL,3,0.986457
985,2017-01-06,45.85,46.720,45.470,46.21,7260197,AAL,4,1.007852
986,2017-01-09,46.01,47.340,45.780,47.08,4739142,AAL,5,1.023256
...,...,...,...,...,...,...,...,...,...
619009,2017-12-22,72.30,72.370,71.790,71.99,1345683,ZTS,247,0.995712
619010,2017-12-26,72.40,72.550,71.900,72.34,792134,ZTS,248,0.999171
619011,2017-12-27,72.59,72.690,72.250,72.45,1159771,ZTS,249,0.998071
619012,2017-12-28,72.49,72.600,72.140,72.39,710499,ZTS,250,0.998620


In [21]:
#identify range of all stocks starting with A

range_ds = df

# get the unique values (rows)
range_ds = range_ds.drop_duplicates(subset = ["Name"])

#All stocks starting with A
range_ds = range_ds.head(59)

range_ds

#first stock
range_ds.head(1)['Name']

#last stock
range_ds.iloc[-1]['Name']


,date,open,high,low,close,volume,Name,Day,payoff
982,2017-01-03,47.2800,47.3400,46.1350,46.300,6737752,AAL,1,0.979272
2241,2017-01-03,115.8000,116.3300,114.7600,116.150,28781865,AAPL,1,1.003022
3500,2017-01-03,170.7800,171.3600,169.3100,170.600,691526,AAP,1,0.998946
4759,2017-01-03,62.9200,63.0300,61.9350,62.410,9328198,ABBV,1,0.991894
6018,2017-01-03,78.5100,83.1000,78.3100,82.610,4134229,ABC,1,1.052223
7277,2017-01-03,38.6334,39.0800,38.3400,39.050,9677316,ABT,1,1.010783
8536,2017-01-03,117.3800,117.8100,115.8200,116.460,2352157,ACN,1,0.992162
9795,2017-01-03,103.4300,104.0300,102.8100,103.480,2277933,ADBE,1,1.000483
11054,2017-01-03,72.6000,74.1292,72.0900,72.505,2687647,ADI,1,0.998691
12313,2017-01-03,45.6900,46.2500,45.5500,46.190,3614876,ADM,1,1.010943


In [9]:
## TESTING ## 

#rounds = 251
#actions = 505
#N = 1000
# ADD OPTIMAL LEARNING RATE EPSILON
opt_lr_eps = math.sqrt(numpy.log(actions)/rounds)
learning_rates = [0, 0.25, 0.5, opt_lr_eps, 0.75, 1, 100]

rotational_payoffs, rotational_totals = generate_rotational_random_payoffs(actions, rounds)

print(rotational_payoffs)
print(rotational_totals)

[[0.64, 0.24, 0.83, 0.54, 0.72], [0.64, 0.24, 0.83, 0.54, 0.72], [0.64, 0.24, 0.83, 0.54, 0.72], [0.72, 0.64, 0.24, 0.83, 0.54], [0.72, 0.64, 0.24, 0.83, 0.54], [0.72, 0.64, 0.24, 0.83, 0.54], [0.72, 0.64, 0.24, 0.83, 0.54], [0.72, 0.64, 0.24, 0.83, 0.54], [0.72, 0.64, 0.24, 0.83, 0.54], [0.72, 0.64, 0.24, 0.83, 0.54], [0.72, 0.64, 0.24, 0.83, 0.54], [0.72, 0.64, 0.24, 0.83, 0.54], [0.72, 0.64, 0.24, 0.83, 0.54], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.54, 0.72, 0.64, 0.24, 0.83], [0.83, 0.54, 0.72, 0.64, 0.24], [0.24, 0.83, 0.54, 0.72, 0.64], [0.24, 0.83, 0.54, 0.72, 0.64], [0.24, 0.83, 0.54, 0.72, 0.64], [0.24, 

## Applying EW algorithm to stock data with payoffs 

In [50]:
def generate_stock_payoffs(dataset, num_actions):
    rounds_list = []
    totals_by_round = []
    total_payoffs = [0 for i in range(num_actions)]
    day = 1
    list_of_dates = []
    while day <= 251:
        temp_ds = dataset[dataset['Day'] == day]
        #print(len(temp_ds['payoff'].tolist()))
        if len(temp_ds['payoff'].tolist()) != num_actions: 
            day += 1
            #print(day)
            continue
        else: 
            list_of_dates.append(temp_ds['date'].iloc[0])
            action_payoffs = temp_ds['payoff'].tolist()
            #print(len(action_payoffs),'action payoffs')
            #print(len(total_payoffs),'total_payoffs')
            #print(day)
            for j in range(num_actions): 
                total_payoffs[j] += action_payoffs[j]
            new_totals = [total_payoffs[i] for i in range(num_actions)]    
            rounds_list.append([action_payoffs[i] for i in range(num_actions)])
            totals_by_round.append(new_totals)
            day += 1

    return rounds_list, totals_by_round, list_of_dates  

In [51]:
actions = 499
N = 1000

#optimal learning rate epsilon 𝜀 = √(ln k / n)
opt_lr_eps = math.sqrt(numpy.log(actions)/rounds)

learning_rates = [0, 0.25, 0.5, opt_lr_eps, 0.75, 1, 100]

max_payoff = 1.167806417674908 #max payoff of all stocks
avg_lr_payoffs = dict()
all_opt_payoffs = []
avg_regret_per_round = dict()
list_of_dates = []
for n in range(N):
    stock_payoffs, stock_totals, date_list = generate_stock_payoffs(df, actions)
    for epsilon in learning_rates:
        sto_payoffs, sto_round_totals = simulate_exponential_weights(stock_payoffs, stock_totals, epsilon, max_payoff)
        sto_regrets = individual_regrets(sto_payoffs, sto_round_totals)
        sto_avg_regrets = sum(sto_regrets) / len(sto_regrets)
        sto_final_regret = sto_regrets[-1]
        if epsilon not in avg_regret_per_round:
            avg_regret_per_round[epsilon] = sto_regrets
        else:
            for i in range(len(avg_regret_per_round[epsilon])):
                avg_regret_per_round[epsilon][i] = ((n * avg_regret_per_round[epsilon][i]) + sto_regrets[i]) / (n + 1)
        
        if epsilon not in avg_lr_payoffs:
            avg_lr_payoffs[epsilon] = [sum(sto_payoffs)]
        else:
            avg_lr_payoffs[epsilon].append(sum(sto_payoffs))
    list_of_dates = date_list
    
    all_opt_payoffs.append(max(sto_round_totals[-1]))
for key, val in avg_regret_per_round.items():
    print("Average ALG regret for epsilon =", key, "on stock data =", val[-1])
for key, val in avg_lr_payoffs.items():
    print("Average ALG payoff for epsilon =", key, "on stock data =", sum(val) / len(val))
print("Average OPT payoff for on stock data =", sum(all_opt_payoffs) / len(all_opt_payoffs) )
print("The list of dates where you are investing are the following...",  list_of_dates)
print("The number of days that you are investing is ",  len(list_of_dates))

499
1
499
2
499
3
499
4
499
5
499
6
499
7
499
8
499
9
499
10
499
11
499
12
499
13
499
14
499
15
499
16
499
17
499
18
499
19
499
20
499
21
499
22
499
23
499
24
499
25
499
26
499
27
499
28
499
29
499
30
499
31
499
32
499
33
499
34
499
35
499
36
499
37
499
38
499
39
499
40
499
41
499
42
499
43
499
44
499
45
499
46
499
47
499
48
499
49
499
50
499
51
499
52
499
53
499
54
499
55
499
56
499
57
499
58
499
59
499
60
499
61
499
62
499
63
499
64
499
65
499
66
499
67
499
68
499
69
499
70
499
71
499
72
499
73
499
74
499
75
499
76
499
77
499
78
499
79
499
80
499
81
499
82
499
83
499
84
499
85
499
86
499
87
499
88
499
89
499
90
499
91
499
92
499
93
499
94
499
95
499
96
499
97
499
98
499
99
499
100
499
101
499
102
499
103
499
104
499
105
499
106
499
107
499
108
499
109
499
110
499
111
499
112
499
113
499
114
499
115
499
116
499
117
499
118
499
119
499
120
499
121
499
122
499
123
499
124
499
125
499
126
499
127
499
128
499
129
499
130
499
131
499
132
499
133
499
134
499
135
499
136
499
137
499
138
499


OverflowError: (34, 'Result too large')

In [49]:
## TESTING ## 

#rounds = 251
actions = 499
#N = 1000
# ADD OPTIMAL LEARNING RATE EPSILON
opt_lr_eps = math.sqrt(numpy.log(actions)/rounds)
learning_rates = [0, 0.25, 0.5, opt_lr_eps, 0.75, 1, 100]

stock_payoffs, stock_totals, list_of_dates = generate_stock_payoffs(df, actions)

print(stock_payoffs)
print(stock_totals)

499
1
499
2
499
3
499
4
499
5
499
6
499
7
499
8
499
9
499
10
499
11
499
12
499
13
499
14
499
15
499
16
499
17
499
18
499
19
499
20
499
21
499
22
499
23
499
24
499
25
499
26
499
27
499
28
499
29
499
30
499
31
499
32
499
33
499
34
499
35
499
36
499
37
499
38
499
39
499
40
499
41
499
42
499
43
499
44
499
45
499
46
499
47
499
48
499
49
499
50
499
51
499
52
499
53
499
54
499
55
499
56
499
57
499
58
499
59
499
60
499
61
499
62
499
63
499
64
499
65
499
66
499
67
499
68
499
69
499
70
499
71
499
72
499
73
499
74
499
75
499
76
499
77
499
78
499
79
499
80
499
81
499
82
499
83
499
84
499
85
499
86
499
87
499
88
499
89
499
90
499
91
499
92
499
93
499
94
499
95
499
96
499
97
499
98
499
99
499
100
499
101
499
102
499
103
499
104
499
105
499
106
499
107
499
108
499
109
499
110
499
111
499
112
499
113
499
114
499
115
499
116
499
117
499
118
499
119
499
120
499
121
499
122
499
123
499
124
499
125
499
126
499
127
499
128
499
129
499
130
499
131
499
132
499
133
499
134
499
135
499
136
499
137
499
138
499


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

